# Python MD5

## Set input array

In [1]:
message = "Mary had a little lamb."

In [2]:
# big endian to little endian function
import struct

def swap32(i):
    return struct.unpack("<I", struct.pack(">I", i))[0]

In [3]:
# get dec values for each character
message_dec = []

for i in range(len(message)):
    message_dec.append(ord(message[i]))
    
print(message_dec)

[77, 97, 114, 121, 32, 104, 97, 100, 32, 97, 32, 108, 105, 116, 116, 108, 101, 32, 108, 97, 109, 98, 46]


In [4]:
# add check byte
message_dec.append(128)

print(message_dec)

[77, 97, 114, 121, 32, 104, 97, 100, 32, 97, 32, 108, 105, 116, 116, 108, 101, 32, 108, 97, 109, 98, 46, 128]


In [5]:
# add zeros up to 56 bytes
padding = 56 - len(message_dec)

for i in range(padding):
    message_dec.append(0)
    
print(message_dec)

[77, 97, 114, 121, 32, 104, 97, 100, 32, 97, 32, 108, 105, 116, 116, 108, 101, 32, 108, 97, 109, 98, 46, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
# add message length in bits
message_dec.append(len(message)*8)

print(message_dec)

[77, 97, 114, 121, 32, 104, 97, 100, 32, 97, 32, 108, 105, 116, 116, 108, 101, 32, 108, 97, 109, 98, 46, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 184]


In [7]:
# add zeros up to 64 bytes
padding = 64 - len(message_dec)

for i in range(padding):
    message_dec.append(0)
    
print(message_dec)

[77, 97, 114, 121, 32, 104, 97, 100, 32, 97, 32, 108, 105, 116, 116, 108, 101, 32, 108, 97, 109, 98, 46, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 184, 0, 0, 0, 0, 0, 0, 0]


In [8]:
# consolidate 64 x 8 bit array into 16 x 32 bit array
counter = 0
M = []

for i in range(16):
    char1 = message_dec[counter]
    char2 = message_dec[counter + 1]
    char3 = message_dec[counter + 2]
    char4 = message_dec[counter + 3]
    chunk = (char1 << 24) + (char2 << 16) + (char3 << 8) + char4
    chunk = swap32(chunk)
    M.append(chunk)
    counter +=4
    
print(M)

[2037539149, 1684105248, 1814061344, 1819571305, 1634476133, 2150523501, 0, 0, 0, 0, 0, 0, 0, 0, 184, 0]


## Set variables

In [9]:
# set offset values
s = []
offset = [[7,12,17,22], [5,9,14,20], [4,11,16,23], [6,10,15,21]]

for i in range(4):
    for j in range(4):
        s.extend(offset[i])

In [10]:
# set k values
import math
K = []

for i in range(64):
    K.append(abs(int(math.sin(i + 1)*(2**32))))

In [11]:
# set initial register values
a0 = 0x67452301 # dec = 1732584193
b0 = 0xefcdab89 # dec = 4023233417
c0 = 0x98badcfe # dec = 2562383102
d0 = 0x10325476 # dec = 271733878

## Main MD5 function

In [12]:
A = a0
B = b0
C = c0
D = d0

x = [0,0,0,0]

for i in range(64):
    
    # roll A,B,C,D
    x[i % 4] = A
    x[(i+1) % 4] = B
    x[(i+2) % 4] = C
    x[(i+3) % 4] = D
    
    if (i < 16):
        F = (x[1] & x[2]) | ((~x[1]) & x[3])
        g = i % 16
    elif (i < 32):
        F = (x[3] & x[1]) | ((~x[3]) & x[2])
        g = (5*i + 1) % 16
    elif (i < 48):
        F = x[1] ^ x[2] ^ x[3]
        g = (3*i + 5) % 16
    else:
        F = x[2] ^ (x[1] | (~x[3]))
        g = (7*i) % 16
    
    F += x[0] + K[i] + M[g]
    F = F % 2**32
    
    # shift your bits
    F = (F << s[i]) % 2**32 | (F >> (32-s[i]) % 2**32)
    
    x[0] = (x[1] + F) % 2**32
    
    if (i%4 == 0): A = x[0]
    if (i%4 == 1): D = x[0]
    if (i%4 == 2): C = x[0]
    if (i%4 == 3): B = x[0]

# consolidate final register values
AA = (a0 + A) % 2**32
BB = (b0 + B) % 2**32
CC = (c0 + C) % 2**32
DD = (d0 + D) % 2**32

MD5 = format(swap32(AA), '08x') + format(swap32(BB), '08x') + format(swap32(CC), '08x') + format(swap32(DD), '08x')

print(MD5)

'ca964b1677d5476ea11eed1e1837c342'

## Compare to built-in Python MD5 library

In [13]:
import hashlib

m = hashlib.md5()
m.update(message.encode('utf-8'))
MD5_python = m.hexdigest()

print(MD5)
print(MD5_python)

if (MD5 == MD5_python):
    print("Success!")
else:
    print("Failure!")

ca964b1677d5476ea11eed1e1837c342
ca964b1677d5476ea11eed1e1837c342
Success!
